# Librerias

In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
import unicodedata
import os

In [2]:
os.chdir(r"/home/jest/Documents/personal/sistemas inteligentes/proyecto/segunda entrega")

# Funciones

In [3]:
def elimina_tildes(s):
    "Elimina tildes y espacio y deja todo formateado en guion bajo"
    s      = re.sub(r'[\W_]+', '_', s).lower().strip('_').strip()
    result = ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))
    return result

# Variables Globales

In [4]:
path_name = 'dataset/SB11_20202.txt'
path_name_model = 'dataset/datos_para_modelo.csv'

## 1. Carga del dataset

In [5]:
%%time
df = pd.read_csv(path_name, '¬', engine='python')
df.columns = [elimina_tildes(x) for x in df.columns.tolist()]

/home/jest/anaconda3/envs/javeriana_tf/lib/python3.9/site-packages/IPython/core/magics/execution.py:1316: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code, glob, local_ns)


CPU times: user 23.1 s, sys: 1.08 s, total: 24.1 s
Wall time: 24.1 s


## 2. Revisión general del Dataset

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397124 entries, 0 to 397123
Data columns (total 81 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   estu_tipodocumento             397124 non-null  object 
 1   estu_nacionalidad              397124 non-null  object 
 2   estu_genero                    397119 non-null  object 
 3   estu_fechanacimiento           397124 non-null  object 
 4   periodo                        397124 non-null  int64  
 5   estu_consecutivo               397124 non-null  object 
 6   estu_estudiante                397124 non-null  object 
 7   estu_pais_reside               397124 non-null  object 
 8   estu_tieneetnia                394806 non-null  object 
 9   estu_depto_reside              397122 non-null  object 
 10  estu_cod_reside_depto          397122 non-null  float64
 11  estu_mcpio_reside              397122 non-null  object 
 12  estu_cod_reside_mcpio         

## 3. Limpieza general del dataset

In [7]:
# Revisión de variables general 
for column_ in df.columns.tolist():
    print('-'*100)
    print(df.groupby(column_).agg(count_ = (column_, 'count')).reset_index().sort_values('count_', ascending=False).head(10))
    print('-'*100)
del column_

----------------------------------------------------------------------------------------------------
  estu_tipodocumento  count_
9                 TI  325409
0                 CC   65444
2                 CR    3857
3                NES    1168
1                 CE     853
7                PEP     322
6                 PE      54
8                 RC      11
4               NUIP       2
5                 PC       2
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
   estu_nacionalidad  count_
9           COLOMBIA  394624
41         VENEZUELA    2311
20    ESTADOS UNIDOS      42
15           ECUADOR      30
18            ESPAÑA      22
13              CUBA       8
5             BRASIL       7
32            PANAMÁ       7
34              PERÚ       7
2          ARGENTINA       6
-----------------------------------------------------------

In [8]:
# Variables para filtro sobre registros
row_filters = [('estu_nacionalidad',  'COLOMBIA'),
               ('estu_tipodocumento',  'TI'),
               ('cole_area_ubicacion','URBANO'),
               ('cole_calendario',  'A'),
               ('cole_naturaleza',  'OFICIAL'),
               ('cole_jornada','MAÑANA')]

In [9]:
print(f'Población inicial: {df.shape[0]}.')
for i in row_filters:
    aux = df.groupby(i[0]).agg(count_ = (i[0], 'count'))
    aux = aux.reset_index().sort_values('count_', ascending=False).head(10)
    print(aux)
    df = df.loc[df[i[0]] == i[1]]
    del aux
    print(f'Total registros con el filtro {i}: {df.shape[0]} registros')
    print('-'*50)
df = df.dropna()
print(f'Despues de no considerar registros con valores nulos, tenemos {df.shape[0]} registros')

Población inicial: 397124.
   estu_nacionalidad  count_
9           COLOMBIA  394624
41         VENEZUELA    2311
20    ESTADOS UNIDOS      42
15           ECUADOR      30
18            ESPAÑA      22
13              CUBA       8
5             BRASIL       7
32            PANAMÁ       7
34              PERÚ       7
2          ARGENTINA       6
Total registros con el filtro ('estu_nacionalidad', 'COLOMBIA'): 394624 registros
--------------------------------------------------
   estu_tipodocumento  count_
9                  TI  325019
0                  CC   65318
2                  CR    3850
3                 NES     223
1                  CE     158
7                 PEP      33
8                  RC      11
6                  PE       7
4                NUIP       2
10                  V       2
Total registros con el filtro ('estu_tipodocumento', 'TI'): 325019 registros
--------------------------------------------------
  cole_area_ubicacion  count_
1              URBANO  272757
0  

# 4. Preparando Dataset para tablero de selección de muestra. 

In [10]:
select_columns = ['estu_genero', 'estu_tieneetnia', 'estu_depto_reside', 'estu_mcpio_reside', 'fami_estratovivienda', 'fami_personashogar', 'fami_cuartoshogar', 'fami_tieneinternet', 'fami_tieneserviciotv', 'fami_tienecomputador', 'fami_tienelavadora', 'fami_tienehornomicroogas', 'fami_tieneautomovil', 'fami_tienemotocicleta', 'fami_tieneconsolavideojuegos', 'cole_bilingue']
target_column = 'punt_global'

In [11]:
# variables puntaje  para tablero
punt_variables=['punt_lectura_critica','punt_matematicas', 'punt_c_naturales',
       'punt_sociales_ciudadanas',  'punt_ingles']

In [12]:
aux = df.groupby(select_columns)
aux = aux.agg(count_ = (target_column, 'count'),
              avg_punt_global = (target_column, 'mean'),
              avg_punt_lectura_critica = ('punt_lectura_critica', 'mean'),
              avg_punt_matematicas = ('punt_matematicas', 'mean'),
              avg_punt_c_naturales = ('punt_c_naturales', 'mean'),
              avg_punt_sociales_ciudadanas = ('punt_sociales_ciudadanas', 'mean'),
              avg_punt_ingles = ('punt_ingles', 'mean'))
aux = aux.reset_index()              
print(aux.shape)
aux.head()

(55062, 23)


,estu_genero,estu_tieneetnia,estu_depto_reside,estu_mcpio_reside,fami_estratovivienda,fami_personashogar,fami_cuartoshogar,fami_tieneinternet,fami_tieneserviciotv,fami_tienecomputador,...,fami_tienemotocicleta,fami_tieneconsolavideojuegos,cole_bilingue,count_,avg_punt_global,avg_punt_lectura_critica,avg_punt_matematicas,avg_punt_c_naturales,avg_punt_sociales_ciudadanas,avg_punt_ingles
0,F,No,AMAZONAS,LETICIA,Estrato 1,1 a 2,Tres,Si,Si,No,...,Si,No,N,1,193.0,46.0,39.0,38.0,29.0,46.0
1,F,No,AMAZONAS,LETICIA,Estrato 1,3 a 4,Cuatro,No,Si,No,...,No,No,N,1,235.0,50.0,43.0,44.0,47.0,58.0
2,F,No,AMAZONAS,LETICIA,Estrato 1,3 a 4,Cuatro,Si,No,No,...,Si,No,N,1,164.0,33.0,37.0,34.0,29.0,28.0
3,F,No,AMAZONAS,LETICIA,Estrato 1,3 a 4,Dos,No,Si,No,...,Si,No,N,1,271.0,51.0,58.0,55.0,57.0,42.0
4,F,No,AMAZONAS,LETICIA,Estrato 1,3 a 4,Dos,Si,Si,No,...,Si,Si,N,1,242.0,43.0,52.0,48.0,51.0,47.0


In [13]:
aux.to_csv('dataset/icfes_dashboard.csv', sep=',')

#### Procedemos a observar el dashboard y seleccionamos la población a estudiar. 

**Resultados Dashboard**
[Dashboard Icfes](https://datastudio.google.com/reporting/93b00516-aed1-440f-a4b1-58c3afdb5dc7)

Despues de analisar el tablero de control del icfes. Observamos que el departamento de Bolivar tiene las siguientes caracteristicas: 

- una mayor proporción de mujeres que de hombres.
- el 46% de la población no tiene acceso a internet. 
- El 45% de la población es estrato 1. 
- EL 70% de las familias estan compuestas por 3 a 7 personas, mientras que los hogares tienen 2 a 3 habitaciones, lo que muestra familiares que conviven en el mismo espacio y puede afectar el puntaje en el icfes. 
- Del top 5 de ciudades por cantidad de Registros, es la ciudad que presenta puntajes más bajos promedios, lo cual resalta el interes de estudiar esta población, en terminos de un indicador de desempeño. 

## 5. Preparando Dataset Para Análisis de modelos.

In [14]:
df = df.assign(desempenio_cat = pd.qcut(df.punt_global, q = 3, labels = False))
def desempenio_name(x):
    if x==0:
        return 'Bajo'
    elif x==1:
        return 'Medio'
    elif x==2:
        return 'Alto'
    else:
        return 'Error'
df = df.assign(desempenio = df.desempenio_cat.apply(desempenio_name))
del desempenio_name

In [15]:
df[['punt_global', 'desempenio_cat', 'desempenio']].head()

,punt_global,desempenio_cat,desempenio
4,225.0,0,Bajo
16,256.0,1,Medio
22,312.0,2,Alto
24,207.0,0,Bajo
42,265.0,1,Medio


In [16]:
print('Desempeño General para la población')
aux = df.groupby('desempenio').agg(count_ = ('desempenio', 'count'))
aux = aux.reset_index()
aux.assign(percent = aux.count_.apply(lambda x: round(x/aux.count_.sum()*100)))

Desempeño General para la población


,desempenio,count_,percent
0,Alto,22853,33
1,Bajo,23829,34
2,Medio,23199,33


In [17]:
print('Desempeño General para la población de Bolivar')
aux = df.loc[df.estu_depto_reside == 'BOLIVAR'].groupby('desempenio').agg(count_ = ('desempenio', 'count'))
aux = aux.reset_index()
aux.assign(percent = aux.count_.apply(lambda x: round(x/aux.count_.sum()*100)))

Desempeño General para la población de Bolivar


,desempenio,count_,percent
0,Alto,808,18
1,Bajo,2479,55
2,Medio,1239,27


In [18]:
df_model = df.loc[df.estu_depto_reside == 'BOLIVAR']

In [19]:
print('Selección de variables')
variables_choose = ['estu_genero', 'estu_tieneetnia', 'estu_depto_reside', 'estu_mcpio_reside', 'fami_estratovivienda', 'fami_personashogar', 'fami_cuartoshogar', 'fami_tieneinternet', 'fami_tieneserviciotv', 'fami_tienecomputador', 'fami_tienelavadora', 'fami_tienehornomicroogas', 'fami_tieneautomovil', 'fami_tienemotocicleta', 'fami_tieneconsolavideojuegos', 'cole_bilingue']
variables_choose.append('punt_global')
variables_choose.extend(['punt_lectura_critica','punt_matematicas', 'punt_c_naturales',
       'punt_sociales_ciudadanas',  'punt_ingles'])
variables_choose.append('desempenio')

Selección de variables


In [20]:
df_model = df_model[variables_choose]

In [21]:
df_model.to_csv(path_name_model, sep = '|' )